## 1. Load NPY and Preprocessing

In [1]:
import numpy as np

# Load point cloud
data = np.load('/home/smrl/cylinder_high.npy')
shiftReal = np.array([0.5, -1, -0.3])
PPm_body = data + shiftReal


## 2. Apply Quaternion Rotation

In [2]:
def quat_to_rotm(q):
    q0, q1, q2, q3 = q
    return np.array([
        [q0**2+q1**2 - q2**2 - q3**2, 2*(q1*q2 - q0*q3), 2*(q1*q3 + q0*q2)],
        [2*(q2*q1 + q0*q3), q0**2 - q1**2 + q2**2 - q3**2, 2*(q2*q3 - q0*q1)],
        [2*(q3*q1 - q0*q2), 2*(q3*q2 + q0*q1), q0**2 - q1**2 - q2**2 + q3**2]
    ])

angle_rad = 17 / 57.92
qo = [np.cos(angle_rad), np.sin(angle_rad), 0, 0]
rotm = quat_to_rotm(qo)
PPmR_body = (rotm @ PPm_body.T).T
PPm_use = PPmR_body


## 3. Non-Homogeneous Polynomial Fitting (Order 6)

In [3]:
import sympy as sp
from scipy.optimize import fsolve

x, y, z = sp.symbols('x y z')
a, b, c = sp.symbols('a b c')
order = 6

def generate_terms(order, homogeneous=False):
    terms = []
    for i in range(order+1):
        for j in range(order+1 - i):
            for k in range(order+1 - i - j):
                if homogeneous and (i + j + k != order):
                    continue
                if not homogeneous and (i + j + k > order):
                    continue
                terms.append(x**i * y**j * z**k)
    return terms

def regression(P, terms):
    M = np.array([[float(t.subs({x: px, y: py, z: pz})) for t in terms] for px, py, pz in P])
    b, *_ = np.linalg.lstsq(M, np.ones(len(P)), rcond=None)
    error = M @ b - 1
    return b, error

TermsA = generate_terms(order, homogeneous=False)
center_shift = np.mean(PPm_use, axis=0)
PPm_shift = PPm_use - center_shift
shiftResidue = shiftReal - center_shift

betaA = sp.symbols(f'betaA0:{len(TermsA)}')
f1 = sum(b * t for b, t in zip(betaA, TermsA))
beta_vals, _ = regression(PPm_use, TermsA)
f1_total = f1.subs(dict(zip(betaA, beta_vals)))
f1_shift = f1_total.subs({x: x+a, y: y+b, z: z+c})
f1_expanded = sp.expand(f1_shift)

poly = sp.Poly(f1_expanded, x, y, z)
coeffsA, monomialA = poly.coeffs(), poly.monoms()
coeffs_unused = [c for c, m in zip(coeffsA, monomialA) if sum(m) != order and sum(m) != 0]
f_coeffsA_norm = sum(sp.sympify(c)**2 for c in coeffs_unused)

f_grad = [sp.diff(f_coeffsA_norm, v) for v in (a, b, c)]
f_func = sp.lambdify((a, b, c), f_grad, 'numpy')

def func_to_solve(vars):
    return np.array(f_func(*vars)).astype(np.float64)

sol = fsolve(func_to_solve, [0, 0, 0])
PPm_shiftA = PPm_use - sol


## 4. Iterative Optimization (Homogeneous Polynomial Fitting)

In [4]:
def regressionShift(P, error, dxyz):
    A = dxyz
    b = -error
    delta, *_ = np.linalg.lstsq(A, b, rcond=None)
    return delta.reshape(-1, 1), b.reshape(-1, 1)

TermsB = generate_terms(order, homogeneous=True)
betaB = sp.symbols(f'betaB0:{len(TermsB)}')
f2 = sum(b * t for b, t in zip(betaB, TermsB))

beta_vals, error_shift = regression(PPm_shift, TermsB)
shift_1 = np.zeros((3,1))
shiftResidue = np.zeros((3,1))
shiftSum = center_shift.reshape(3,1)
PPm_shift = PPm_use - center_shift

for i in range(60):
    d1 = np.array([[float(sp.diff(f2, x).subs({x: px, y: py, z: pz, **dict(zip(betaB, beta_vals))})) for px, py, pz in PPm_shift]])[0]
    d2 = np.array([[float(sp.diff(f2, y).subs({x: px, y: py, z: pz, **dict(zip(betaB, beta_vals))})) for px, py, pz in PPm_shift]])[0]
    d3 = np.array([[float(sp.diff(f2, z).subs({x: px, y: py, z: pz, **dict(zip(betaB, beta_vals))})) for px, py, pz in PPm_shift]])[0]
    dxyz = np.vstack((d1, d2, d3)).T

    shift, _ = regressionShift(PPm_shift, error_shift, dxyz)
    shift_1 = 0.85 * shift + 0.66 * shift_1
    shiftResidue += shift_1
    shiftSum -= shift_1
    PPm_shift += shift_1.T
    beta_vals, error_shift = regression(PPm_shift, TermsB)


KeyboardInterrupt: 

## 5. Quaternion-Based Rotation Optimization

In [ ]:
from scipy.optimize import minimize
q0, q1, q2, q3 = sp.symbols('q0 q1 q2 q3', real=True)
q = [q0, q1, q2, q3]
Rq_sym = quat_to_rotm(q)
xr = Rq_sym[0,0]*x + Rq_sym[0,1]*y + Rq_sym[0,2]*z
yr = Rq_sym[1,0]*x + Rq_sym[1,1]*y + Rq_sym[1,2]*z
zr = Rq_sym[2,0]*x + Rq_sym[2,1]*y + Rq_sym[2,2]*z

f4_rotated = f2.subs(dict(zip(betaB, beta_vals)))
f4_rotated = f4_rotated.subs({x: xr, y: yr, z: zr})

poly_rot = sp.Poly(f4_rotated, x, y, z)
coeffsB, monomialB = poly_rot.coeffs(), poly_rot.monoms()
monList = [(0,6,0), (2,4,0), (4,2,0), (6,0,0), (0,0,6)]
coeffsB_unused = [c for c, m in zip(coeffsB, monomialB) if tuple(m) not in monList]
f_coeffsB_norm = sum(sp.sympify(c)**2 for c in coeffsB_unused)

f_obj = sp.lambdify((q0, q1, q2, q3), f_coeffsB_norm, 'numpy')
objective = lambda q: f_obj(*q)
constraint = {'type': 'eq', 'fun': lambda q: np.sum(np.square(q)) - 1}

res = minimize(objective, [1, 0, 0, 0], constraints=constraint)
qOpt = res.x / np.linalg.norm(res.x)

Rq4 = quat_to_rotm([qOpt[0], -qOpt[1], -qOpt[2], -qOpt[3]])
PPm_shiftB = (Rq4 @ PPm_shift.T).T


## 6. Final Visualization

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

f4_substituted = f4_rotated.subs({q0: qOpt[0], q1: qOpt[1], q2: qOpt[2], q3: qOpt[3]})
f4_numeric = sp.lambdify((x, y, z), f4_substituted - 1, 'numpy')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(PPm_shiftB[:,0], PPm_shiftB[:,1], PPm_shiftB[:,2], s=1, c=PPm_shiftB[:,2], cmap='jet')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=30, azim=45)
ax.set_title("Final Aligned Point Cloud and Surface")
plt.axis('equal')
plt.show()
